In [1]:
from planet4 import io, region_data, markings
from p4_tools import get_final_markings_counts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import root, curve_fit, leastsq
import pdb
import shapely.geometry as shp
import fiona as fio
from shapely import affinity
from shapely.ops import cascaded_union, unary_union

structure mostly taken from Fans_Blotches_over_time_GP_updated

In [2]:
''' defining input: input is a list of filenames as strings ('filename.csv'). 
input can contain information for one or more regions. For 1 region, input should contain that region's fan file, 
followed by the season 2 metadata for that region, followed by the season 3 metadata for that region, 
followed by that region's blotches file. For more than 1 region, add more filenames in the same order. 
'''
_input_Giza = ['giza_fans.csv','giza_season2_metadata.csv','giza_season3_metadata.csv','giza_blotches.csv']
_input_Inca = ['Inca_seasons23_fans.csv','inca_season2_metadata.csv','inca_season3_metadata.csv','Inca_seasons23_blotches.csv']
_input_Ithaca = ['ithaca_fans.csv','ithaca_season2_metadata.csv','ithaca_season3_metadata.csv','ithaca_blotches.csv']
_input_Manh = ['manhattan_fans.csv','manhattan_season2_metadata.csv','manhattan_season3_metadata.csv','manhattan_blotches.csv']

def get_regions(_input_):
    '''get_regions creates a dataframe to help step through the processes below. 
    note that fan and blotch file names must be identical to the ones following 
    these "if" statements for the function to recognize them properly'''
    regions = pd.DataFrame(np.zeros((4,1)))
    if _input_[0]=='giza_fans.csv':
        regions.loc[0] = _input_[0]
        regions.loc[1] = _input_[1]
        regions.loc[2] = _input_[2]
        regions.loc[3] = _input_[3]
        regions.loc[4] = 'Giza'
        fan_start = 10
        blotch_start = 16
    elif _input_[0]=='Inca_seasons23_fans.csv':
        regions.loc[0] = _input_[0]
        regions.loc[1] = _input_[1]
        regions.loc[2] = _input_[2]
        regions.loc[3] = _input_[3]
        regions.loc[4] = 'Inca City'
        fan_start = 494
        blotch_start = 8
    elif _input_[0]=='ithaca_fans.csv':
        regions.loc[0] = _input_[0]
        regions.loc[1] = _input_[1]
        regions.loc[2] = _input_[2]
        regions.loc[3] = _input_[3]
        regions.loc[4] = 'Ithaca'
        fan_start = 11
        blotch_start = 2000
    elif _input_[0]=='manhattan_fans.csv':
        regions.loc[0] = _input_[0]
        regions.loc[1] = _input_[1]
        regions.loc[2] = _input_[2]
        regions.loc[3] = _input_[3]
        regions.loc[4] = 'Manhattan'
        fan_start = 811
        blotch_start = 16626
    return regions,fan_start,blotch_start

# fan_start and blotch_start are arbitrary indexes that I found to give good example plots. I tried random integers and settled on values that gave nicely clustered objects.
regions,fan_start,blotch_start = get_regions(_input_Giza)
regions

,0
0,giza_fans.csv
1,giza_season2_metadata.csv
2,giza_season3_metadata.csv
3,giza_blotches.csv
4,Giza


In [3]:
# read out metadata for season 2 and 3
s2_meta = pd.read_csv(regions[0][1])
s3_meta = pd.read_csv(regions[0][2])

# read out actual markings for fans and blotches, both seasons combined
fans = pd.read_csv(regions[0][0])
blotches = pd.read_csv(regions[0][3])

# find unique obsids in the fans and blotches catalogs
fimg_names = fans.image_name.unique()
bimg_names = blotches.image_name.unique()

# add column for season flag and validity of marking
fans['season'] = 0
fans['valid_marking'] = True
blotches['season'] = 0
blotches['valid_marking'] = True
blotches['area'] = np.pi * blotches.radius_1 * blotches.radius_2 / 4

# define column for season flag
fans.season[fans.obsid.str[5] == '1'] = 2
fans.season[fans.obsid.str[5] == '2'] = 3
blotches.season[blotches.obsid.str[5] == '1'] = 2
blotches.season[blotches.obsid.str[5] == '2'] = 3

# find what size fans should be removed for fair comparison
min_fan_pixels = fans.distance.min()
min_bl_area = blotches.area.min()

max_scale = np.max( (s2_meta.map_scale.max(), s3_meta.map_scale.max() ) )
print('maximum map_scale:',  max_scale, '; min_fan_marking:', min_fan_pixels, '; min_bl_area:', min_bl_area)

maximum map_scale: 1.0 ; min_fan_marking: 10.0 ; min_bl_area: 78.5398163397


/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying

In [ ]:
# if the minimal fan marking tool is = 10 pixels at max_bin = 4, it will correspond to  
# min_fan_pixels * max_binning / image_binning
s2_meta['min_fan'] = min_fan_pixels * s2_meta.map_scale.max() // s2_meta.map_scale + 1
s3_meta['min_fan'] = min_fan_pixels * s3_meta.map_scale.max() // s3_meta.map_scale + 1

# if the minimal blotch marking tool is = 80 sq. pixels at max_bin = 4, it will correspond to  
# min_bl_area * max_binning^2 / image_binning^2
s2_meta['min_bl'] = min_bl_area * s2_meta.map_scale.max()**2 // s2_meta.map_scale**2 + 1
s3_meta['min_bl'] = min_bl_area * s3_meta.map_scale.max()**2 // s3_meta.map_scale**2 + 1
s2_meta.head(10)

,binning,invalids,l_s,line_samples,lines,map_scale,obsid,path,real_area,min_fan,min_bl
0,4,0.925383,185.508023,41899,45359,0.50,ESP_011447_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,3.545212e+07,21.0,315.0
1,4,0.881249,185.552493,10318,32810,1.00,ESP_011448_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,4.020121e+07,11.0,79.0
2,2,0.900835,200.540424,26204,38043,0.50,ESP_011777_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,2.471390e+07,21.0,315.0
3,2,0.898351,203.616291,25209,38913,0.50,ESP_011843_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,2.492830e+07,21.0,315.0
4,4,0.804329,223.749771,23631,35954,0.25,ESP_012265_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,1.039049e+07,41.0,1257.0
5,2,0.811829,227.593263,25187,34687,0.25,ESP_012344_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,1.027484e+07,41.0,1257.0
6,2,0.826119,247.714498,13446,23239,0.50,ESP_012753_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,1.358324e+07,21.0,315.0
7,2,0.598537,251.818782,15035,5288,0.50,ESP_012836_0850,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,7.979588e+06,21.0,315.0
8,2,0.816033,252.265089,13164,17244,0.50,ESP_012845_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,1.044013e+07,21.0,315.0
9,4,0.845716,221.182466,33087,41346,0.25,ESP_012212_0950,/Users/klay6683/Dropbox/data/hirise/labels/ESP...,1.319142e+07,41.0,1257.0


In [ ]:
# mark "valid_marking" key to be False for blotch markings smaller than min_bl for that image
for i in range(len(blotches)):
    if (blotches.season[i] == 2): 
        nr_image =  np.where(s2_meta.obsid == blotches.obsid[i])[0][0]        
        min_b = s2_meta.min_bl[ nr_image  ]         
        if (blotches.area[i] < min_b ):
            blotches.valid_marking[i] = False
    if (blotches.season[i] == 3): 
        nr_image =  np.where(s3_meta.obsid == blotches.obsid[i])[0][0]        
        min_bl = s3_meta.min_bl[ nr_image  ]         
        if (blotches.area[i] < min_b ):
            blotches.valid_marking[i] = False
            
# mark "valid_marking" key to be False for fan markings smaller than min_fan for that image
for i in range(len(fans)):
    if (fans.season[i] == 2): 
        nr_image =  np.where(s2_meta.obsid == fans.obsid[i])[0][0]        
        min_f = s2_meta.min_fan[ nr_image  ]         
        if (fans.distance[i] < min_f ):
            fans.valid_marking[i] = False
    if (fans.season[i] == 3): 
        nr_image =  np.where(s3_meta.obsid == fans.obsid[i])[0][0]        
        min_f = s3_meta.min_fan[ nr_image  ]         
        if (fans.distance[i] < min_f ):
            fans.valid_marking[i] = False

fans.head(10)

/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


next cell creates dataframe for all objects in both seasons, puts both seasons of metadata together, and adds columns for coverage

In [ ]:
#only valid markings are selected for the objects dataframe
objects = pd.DataFrame({'name':pd.concat((fans.image_name[fans.valid_marking==True],blotches.image_name[blotches.valid_marking==True])),'type':np.zeros(len(fans.valid_marking[fans.valid_marking==True])+len(blotches.valid_marking[blotches.valid_marking==True])),'long':pd.concat((fans.distance[fans.valid_marking==True],blotches.radius_1[blotches.valid_marking==True])),'short':pd.concat((fans.spread[fans.valid_marking==True],blotches.radius_2[blotches.valid_marking==True])),'x':pd.concat((fans.image_x[fans.valid_marking==True],blotches.image_x[blotches.valid_marking==True])),'y':pd.concat((fans.image_y[fans.valid_marking==True],blotches.image_y[blotches.valid_marking==True])),'angle':pd.concat((fans.angle[fans.valid_marking==True],blotches.angle[blotches.valid_marking==True])),'binning':np.zeros(len(fans.valid_marking[fans.valid_marking==True])+len(blotches.valid_marking[blotches.valid_marking==True])),'l_s':np.zeros(len(fans.valid_marking[fans.valid_marking==True])+len(blotches.valid_marking[blotches.valid_marking==True])),'x_angle':pd.concat((fans.x_angle[fans.valid_marking==True],blotches.x_angle[blotches.valid_marking==True])),'y_angle':pd.concat((fans.y_angle[fans.valid_marking==True],blotches.y_angle[blotches.valid_marking==True]))}).reset_index(drop=True)
img_unique = objects.name.unique()
meta = pd.concat((s2_meta,s3_meta)).reset_index(drop=True)
meta['coverage'] = 0
meta['coverage_redundant'] = 0
meta['season'] = 0
meta.iloc[:len(s2_meta),13] = 2
meta.iloc[len(s2_meta):,13] = 3

#these two should be the same
print(len(objects))
print(len(fans.valid_marking[fans.valid_marking==True])+len(blotches.valid_marking[blotches.valid_marking==True]))

#add column for binning and l_s
for k in range(img_unique.size):
    objects.iloc[np.where(objects.name==img_unique[k])[0],2] = np.asarray(meta.iloc[np.where(meta.obsid==img_unique[k])[0],2])
    objects.iloc[np.where(objects.name==img_unique[k])[0],1] = np.asarray(meta.iloc[np.where(meta.obsid==img_unique[k])[0],0])
    objects.loc[:len(fans.valid_marking[fans.valid_marking==True]),'type'] = 'f'
    objects.loc[len(fans.valid_marking[fans.valid_marking==True]):,'type'] = 'b'
    
objects.head(10)

next cell creates list to be filled with shapely polygons. for fans, shapes are given by an isosceles triangle with a 13-point-approximated semi circle on top (ice cream cone), rotated about the bottom of the cone according to "angle". For blotches, shapes are given by a circle scaled according to the two radii "radius_1" and "radius_2", now "long" and "short," and rotated about center according to "angle."

In [ ]:
collection = [None] * len(objects)
points = [None] * 14
areas = np.zeros(len(objects))

#re-sort objects according to image, so that blotches and fans for the same image won't be split up
objects = objects.sort_values(by='name').reset_index(drop=True)
meta = meta.sort_values(by='obsid').reset_index(drop=True)

#loop through all objects
for k in range(len(objects)):
    #select for fans vs. blotches
    if objects.type[k]=='f':
        #multiply length of fan by binning
        objects.iloc[k,3] = objects.iloc[k,3]*float(objects.iloc[k,1])
        #get radius of the semi-circle
        r = objects.long[k]*np.tan((objects.short[k]/2.)*np.pi/180.)*(1./np.sqrt(2))*(1./np.sin(((objects.short[k]/2.)+45)*np.pi/180.))*np.sin((90-(objects.short[k]/2.))*np.pi/180.)
        red_ax = objects.long[k] - r
        #bottom of cone
        points[0] = (objects.x[k],objects.y[k])
        #second point (part of triangle, base of semi-circle), going ccw from here
        points[1] = (objects.x[k] + r,objects.y[k] + red_ax)
        #next 11 points are on semi-cirlce
        points[2] = (objects.x[k] + r*np.cos(15.*np.pi/180.),objects.y[k] + r*np.sin(15.*np.pi/180.) + red_ax)
        points[3] = (objects.x[k] + r*np.cos(30.*np.pi/180.),objects.y[k] + r*np.sin(30.*np.pi/180.) + red_ax)
        points[4] = (objects.x[k] + r*np.cos(45.*np.pi/180.),objects.y[k] + r*np.sin(45.*np.pi/180.) + red_ax)
        points[5] = (objects.x[k] + r*np.cos(60.*np.pi/180.),objects.y[k] + r*np.sin(60.*np.pi/180.) + red_ax)
        points[6] = (objects.x[k] + r*np.cos(75.*np.pi/180.),objects.y[k] + r*np.sin(75.*np.pi/180.) + red_ax)
        points[7] = (objects.x[k] + r*np.cos(90.*np.pi/180.),objects.y[k] + r*np.sin(90.*np.pi/180.) + red_ax)
        points[8] = (objects.x[k] + r*np.cos(105.*np.pi/180.),objects.y[k] + r*np.sin(105.*np.pi/180.) + red_ax)
        points[9] = (objects.x[k] + r*np.cos(120.*np.pi/180.),objects.y[k] + r*np.sin(120.*np.pi/180.) + red_ax)
        points[10] = (objects.x[k] + r*np.cos(135.*np.pi/180.),objects.y[k] + r*np.sin(135.*np.pi/180.) + red_ax)
        points[11] = (objects.x[k] + r*np.cos(150.*np.pi/180.),objects.y[k] + r*np.sin(150.*np.pi/180.) + red_ax)
        points[12] = (objects.x[k] + r*np.cos(165.*np.pi/180.),objects.y[k] + r*np.sin(165.*np.pi/180.) + red_ax)
        #last point, third point of triangle and other base point for semi-circle, connects to first point
        points[13] = (objects.x[k] + r*np.cos(180.*np.pi/180.),objects.y[k] + r*np.sin(180.*np.pi/180.) + red_ax)
        #define these points as a polygon in in collections
        collection[k] = shp.Polygon(points[:])
        #rotate polygon about first point according to angle
        collection[k] = affinity.rotate(collection[k],angle=objects.angle[k] - 90.,origin=points[0])
        #record area of each object
        areas[k] = collection[k].area
    else:
        #multiply both radii of blotch by binning
        objects.iloc[k,3] = objects.iloc[k,3]*float(objects.iloc[k,1])
        objects.iloc[k,5] = objects.iloc[k,5]*float(objects.iloc[k,1])
        #define unit circle at center of blotch
        circle = shp.point.Point(objects.x[k],objects.y[k]).buffer(1)
        #stretch unit circle according to semi-major and semi-minor axes of ellipse
        collection[k] = affinity.scale(circle,objects.long[k],objects.short[k])
        collection[k] = affinity.rotate(collection[k],angle=objects.angle[k])
        #record area
        areas[k] = collection[k].area

#example plots of fans 
plt.subplot(121)
plt.plot(collection[fan_start].exterior.xy[0],collection[fan_start].exterior.xy[1])
plt.plot(collection[fan_start+1].exterior.xy[0],collection[fan_start+1].exterior.xy[1])
plt.plot(collection[fan_start+2].exterior.xy[0],collection[fan_start+2].exterior.xy[1])
plt.axis('equal')

#example plots of blotches
plt.subplot(122)
plt.plot(collection[len(objects) - (blotch_start)].exterior.xy[0],collection[len(objects) - (blotch_start)].exterior.xy[1])
plt.plot(collection[len(objects) - (blotch_start+1)].exterior.xy[0],collection[len(objects) - (blotch_start+1)].exterior.xy[1])
plt.plot(collection[len(objects) - (blotch_start+2)].exterior.xy[0],collection[len(objects) - (blotch_start+2)].exterior.xy[1])
plt.axis('equal')
plt.show()

then, unions are calculated for the objects in each image

In [ ]:
#loop through images
for k in range(len(meta)):
    if k==0:
        #get number of objects for a particular image
        ind = len(np.where(objects.name==meta.obsid[k])[0])
        #index to that number for the first iteration, calculate union of all objects in that image
        meta.iloc[k,11] = unary_union(collection[:ind]).area
    else:
        #recall previous index, this will be the lower range for all other iterations
        ind_minus = ind
        #upper range is lower range plus number of objects in current image
        ind = ind_minus + len(np.where(objects.name==meta.obsid[k])[0])
        #index from lower to upper range, calculate union of all objects in that image
        meta.iloc[k,11] = unary_union(collection[ind_minus:ind]).area
    #calculate sum of areas of all objects in an image for comparison (not paying attention to overlap), store as coverage_redundant
    meta.iloc[k,12] = np.sum(areas[np.where(objects.name==meta.obsid[k])[0]])
    
#example plot of objects overlapping
plt.subplot(121)
plt.plot(collection[fan_start].exterior.xy[0],collection[fan_start].exterior.xy[1])
plt.plot(collection[fan_start+1].exterior.xy[0],collection[fan_start+1].exterior.xy[1])
plt.plot(collection[fan_start+2].exterior.xy[0],collection[fan_start+2].exterior.xy[1])
plt.axis('equal')

#example plot of union of  those objects
union = unary_union(collection[fan_start:fan_start+3])
plt.subplot(122)
plt.plot(union.exterior.xy[0],union.exterior.xy[1])
plt.axis('equal')
plt.show()
#print sum of areas of these objects
print(np.sum(areas[fan_start:fan_start+3]))

#print area of union of these objects, should be less than sum of areas
print(unary_union(collection[fan_start:fan_start+3]).area)


one plot is produced for each season

In [ ]:
plt.figure()
plt.subplot(121)
#dataframes are sorted by image name, meaning season 2 will be first and season 3 second, so indexing to the size of the s2_meta array will ensure only season 2 is plotted here
plt.scatter(meta.l_s[:len(s2_meta)],meta.coverage[:len(s2_meta)])
#set same x axis for both, scale according to min/max l_s
plt.xlim((min(meta.l_s)-10,max(meta.l_s)+10))
#set same y axis for both, scale according to maximum coverage value
plt.ylim((0,max(meta.coverage)*1.1))
plt.title('S2 Covered Area - {}'.format(regions[0][4]))
plt.xlabel('L_s (degrees)')
plt.ylabel('Covered Area (pixels*binning)')
plt.subplot(122)
plt.scatter(meta.l_s[len(s2_meta):],meta.coverage[len(s2_meta):])
plt.xlim((min(meta.l_s)-10,max(meta.l_s)+10))
plt.ylim((0,max(meta.coverage)*1.1))
plt.title('S3 Covered Area - {}'.format(regions[0][4]))
plt.xlabel('L_s (degrees)')
plt.ylabel('Covered Area (pixels*binning)')
plt.show()